# Easy (+0.1)

In [1]:
def func(numbers):
    result_dict = {"mean": ..., "median": ..., "mode": ...}

    # Calculate the mean value
    result_dict["mean"] = sum(numbers) / len(numbers)

    # Calculate the median value
    sorted_numbers = sorted(numbers)
    length = len(sorted_numbers) 
    if length % 2 == 0:
        mid1 = sorted_numbers[length // 2 - 1]
        mid2 = sorted_numbers[length // 2]
        result_dict["median"] = (mid1 + mid2) / 2
    else:
        result_dict["median"] = sorted_numbers[length // 2]

    # Calculate the mode value
    mode = None
    mode_value = -1
    counts = {}
    for num in numbers:
        counts[num] = counts.get(num, 0) + 1
        if counts[num] > mode_value:
            mode_value, mode = counts[num], num
    result_dict["mode"] = mode
    
    return result_dict

In [2]:
func([0, 1, 1, 10, 5, 4, 3])

{'mean': 3.4285714285714284, 'median': 3, 'mode': 1}

# Medium (+0.4)

In [3]:
def parse_csv(file_content: str) -> list[dict]:
    
    heroes_list = list()
    
    with open(file_content, 'r') as file:
        columns = file.readline().strip().split(',')
        columns[0] = 'index'
        for line in file:
            values = line.split(',', 7)
            roles_string_parsed = ""
            for char in values[7]:
                if char in ["\"", "\'", ']', '[', ',', '\n']:
                    continue
                else:
                    roles_string_parsed += char
            values[7] = roles_string_parsed.split(' ')
            
            hero_dict = dict(zip(columns, values))
            hero_dict['index'] = int(hero_dict['index'])
            hero_dict['id'] = int(hero_dict['id'])
            hero_dict['legs'] = int(hero_dict['legs'])
            
            heroes_list.append(hero_dict)

    return heroes_list

In [4]:
data = parse_csv("../data/dota_hero_stats.csv")

In [5]:
data[0:3]

[{'index': 0,
  'attack_type': 'Melee',
  'id': 1,
  'legs': 2,
  'localized_name': 'Anti-Mage',
  'name': 'npc_dota_hero_antimage',
  'primary_attr': 'agi',
  'roles': ['Carry', 'Escape', 'Nuker']},
 {'index': 1,
  'attack_type': 'Melee',
  'id': 2,
  'legs': 2,
  'localized_name': 'Axe',
  'name': 'npc_dota_hero_axe',
  'primary_attr': 'str',
  'roles': ['Initiator', 'Durable', 'Disabler', 'Jungler']},
 {'index': 2,
  'attack_type': 'Ranged',
  'id': 3,
  'legs': 4,
  'localized_name': 'Bane',
  'name': 'npc_dota_hero_bane',
  'primary_attr': 'int',
  'roles': ['Support', 'Disabler', 'Nuker', 'Durable']}]

## Find a hero with the largest value of ```legs```

In [6]:
def get_a_hero_with_a_maximum_number_of_legs(heroes):
    max_legs = 0
    hero_index = None
    for index, hero in enumerate(data):
        if hero['legs'] > max_legs:
            max_legs = hero['legs']
            hero_index = index
    return heroes[hero_index]

In [7]:
get_a_hero_with_a_maximum_number_of_legs(heroes=data)

{'index': 59,
 'attack_type': 'Melee',
 'id': 61,
 'legs': 8,
 'localized_name': 'Broodmother',
 'name': 'npc_dota_hero_broodmother',
 'primary_attr': 'agi',
 'roles': ['Carry', 'Pusher', 'Escape', 'Nuker']}

## ANSWER: 8

# Hard (+ в карму)

1. Each possible role should be represented by a column (using one-hot encoding). <h3><center> $\rho(x_i, x_j) = |x_{i, \text{legs}} - x_{j, \text{legs}}| + \sum_k I(x_{i, k} \neq x_{j, k})$ </center></h3> where the last sum iterates through all features except ``legs`` (after one-hot encoding is implied).
2. Since $0 \leq$ ```legs``` $\leq 8$, MinMaxScaler isn't necessary. However, to make it less impactful, let's apply the following:
<h3><center> $legs_i := \frac{legs_i}{3}$ <center></h3>

In [8]:
def scaler(data):
    for idx in range(len(data)):
        data[idx]['legs'] /= 3
    return data

In [9]:
data = scaler(data)

In [10]:
def add_one_hot_encoding(data):
    for idx in range(len(data)):
        for role in data[idx]['roles']:
            data[idx][role] = 1
    return data

In [11]:
data = add_one_hot_encoding(data)

In [12]:
def compute_the_distance(hero1: dict, hero2: dict) -> float: 
    # computes the distance for two heroes according to the formula above
    distance = abs(hero1['legs'] - hero2['legs'])
    for feature in ['attack_type', 'primary_attr']:
        if hero1[feature] != hero2[feature]:
            distance += 1
    for role in hero1['roles']:
        distance += abs(hero1[role] - hero2.get(role, 0))

    for role in hero2['roles']:
        distance += abs(hero2[role] - hero1.get(role, 0))

    return distance

In [13]:
def find_two_the_closest_heroes(data):
    dist_min = 1e9
    hero1_index, hero2_index = None, None
    
    for i in range(len(data)):
        for j in range(i + 1, len(data)):
            dist = compute_the_distance(data[i], data[j])
            if dist < dist_min:
                dist_min = dist
                hero1_index = i
                hero2_index = j
    return dist_min, data[hero1_index], data[hero2_index]

In [14]:
find_two_the_closest_heroes(data)

(0.0,
 {'index': 4,
  'attack_type': 'Ranged',
  'id': 5,
  'legs': 0.6666666666666666,
  'localized_name': 'Crystal Maiden',
  'name': 'npc_dota_hero_crystal_maiden',
  'primary_attr': 'int',
  'roles': ['Support', 'Disabler', 'Nuker', 'Jungler'],
  'Support': 1,
  'Disabler': 1,
  'Nuker': 1,
  'Jungler': 1},
 {'index': 88,
  'attack_type': 'Ranged',
  'id': 90,
  'legs': 0.6666666666666666,
  'localized_name': 'Keeper of the Light',
  'name': 'npc_dota_hero_keeper_of_the_light',
  'primary_attr': 'int',
  'roles': ['Support', 'Nuker', 'Disabler', 'Jungler'],
  'Support': 1,
  'Nuker': 1,
  'Disabler': 1,
  'Jungler': 1})